In [1]:
from scipy import stats
import random
import time
import matplotlib.pyplot as plt
import pylob as lob
from IPython.display import clear_output

In [2]:
def generate_orders(n: int):
    result = list()
    
    for _ in range(n):
    
        midprice = 110_000
    
        n_ask_limits = stats.poisson.rvs(1000)
        n_bid_limits = stats.poisson.rvs(1000)
    
        ask_limits_price = stats.expon.rvs(loc=midprice, scale=1, size=n_ask_limits)
        bid_limits_price = -stats.expon.rvs(loc=midprice, scale=1, size=n_bid_limits) + 2*midprice
    
        ask_limits_quantities = stats.uniform.rvs(loc=1, scale=100, size=n_ask_limits)
        bid_limits_quantities = stats.uniform.rvs(loc=1, scale=100, size=n_bid_limits)
    
        ask_limits_params = [lob.OrderParams(lob.OrderSide.ASK, p, q) for (p, q) in zip(ask_limits_price, ask_limits_quantities)]
        bid_limits_params = [lob.OrderParams(lob.OrderSide.BID, p, q) for (p, q) in zip(bid_limits_price, bid_limits_quantities)]
    
        n_markets = stats.poisson.rvs(100)
    
        markets_price = stats.norm.rvs(loc=midprice, scale=2, size=n_markets)
        markets_quantities = stats.uniform.rvs(loc=1, scale=100, size=n_markets)
        markets_bid_or_ask = [random.choice((lob.OrderSide.BID, lob.OrderSide.ASK)) for _ in range(n_markets)]
    
        markets_params = [lob.OrderParams(s, p, q) for (s, p, q) in zip(markets_bid_or_ask, markets_price, markets_quantities)]
    
        orders = ask_limits_params + bid_limits_params + markets_params
        random.shuffle(orders)
        
        result.append(orders)
        
    return result

In [3]:
def simulate(orders):
    ob = lob.OrderBook('Simulation')

    for o in orders:
        ob.process_many(o)
        print(ob.view())
        time.sleep(0.7)
        clear_output(wait=True)

In [4]:
orders = generate_orders(10)
simulate(orders)

   [ORDER-BOOK Simulation]

   ...(518 more asks)
 - 110000.09 | 098 | 04539.43 | 499337708.5487
 - 110000.08 | 109 | 05966.57 | 656323177.3256
 - 110000.07 | 109 | 05584.86 | 614334990.9402
 - 110000.06 | 112 | 05516.08 | 606769130.9648
 - 110000.05 | 101 | 04889.59 | 537855144.4795
 - 110000.04 | 094 | 04614.88 | 507636984.5952
 - 110000.03 | 046 | 02580.36 | 283839677.4108
 - 110000.02 | 005 | 00259.58 | 28553805.1916
 - 110000.01 | 001 | 00048.87 | 5375700.4887
 - 110000.00 | 002 | 00143.54 | 15789400.0000
 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
 - 109999.99 | 003 | 00188.59 | 20744898.1141
 - 109999.98 | 010 | 00471.38 | 51851790.5724
 - 109999.97 | 064 | 03428.32 | 377115097.1504
 - 109999.96 | 099 | 04945.73 | 544030102.1708
 - 109999.95 | 080 | 04455.09 | 490059677.2455
 - 109999.94 | 082 | 04279.75 | 470772243.2150
 - 109999.93 | 105 | 05322.80 | 585507627.4040
 - 109999.92 | 096 | 05406.31 | 594693667.4952
 - 109999.91 | 086 | 04290.32 | 471934813.8712
 - 109999.90 | 09